Introduction a SQLAlchemy 
=======================

## ¿Qué es SQLAlchemy?

* SQLAlchemy es una librería de Python que provee de un interface para bases de datos relacionales como:
    - Oracle
    - MySQL
    - PostgreSQL
    - DB2
    - SQLite
* SQLAlchemy incluye además un object-relational mapper (ORM).

## ¿Por qué usar SQLAlchemy?

* Portabilidad y abstracción: el mismo código se puede emplear para distinstas bases de datos
* Seguridad: no es necesario SQL injections
* Orientado a objetos: el trabajo se realiza con objetos y no con tablas 
* Rendimiento: las consultas pueden reutilizarse
* Flexibilidad: puede realizarse casi cualquier cosa

## Instalación

In [0]:
!pip install sqlalchemy

In [0]:
!conda install sqlalchemy

## Conexión con bases de datos

Para poder interactuar con una base de datos es necesario conocer las credenciales para poder acceder.

La sintaxis de la ruta de conexión a una base de datos utiliza la siguiente sintaxis:

```<dialecto><controlador>://<usuario>:<contraseña>@<ruta del servidor>:<puerto>/<base de datos>```

Para mayor información, consultar en el [link](http://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

## Tipos de datos

La lista completa está disponible en el [link](https://docs.sqlalchemy.org/en/13/core/type_basics.html).

| SQLAlchemy | Python | SQL |
| ---------- | ---------- |  ---------- |
| BigInteger | int | BIGINT|
| Boolean | bool | BOOLEAN or SMALLINT |
|Date | datetime.date | Date (SQLite: String)|
| DateTime | datetime.datetime | DATETIME (SQLite: String)|
| Enum | str | ENUM or VARCHAR| 
| Float | float or Decimal | FLOAT or REAL|
|Integer |int | Integer|
|Interval | datetime.timedelta | INTERVAL or DATE from epoch| 
| LargeBinary | byte | BLOB or BYTEA |
| Numeric | decimal.Decimal | NUMERIC or DECIMAL |
| Unicode | unicode | UNICODE or VARCHAR |
| Text | str | CLOB or TEXT | 
| Time | datetime.time | DATETIME | 

### Inicio

In [0]:
import sqlalchemy

# Connexion a la base de datos
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:',echo=True)

In [0]:
!ls -la

total 16
drwxr-xr-x 1 root root 4096 May 31 16:17 .
drwxr-xr-x 1 root root 4096 Jun  4 07:34 ..
drwxr-xr-x 1 root root 4096 May 31 16:17 .config
drwxr-xr-x 1 root root 4096 May 31 16:17 sample_data


### Creación de Tablas

Para la creación de tablas se usa un objeto base: 

In [0]:
from sqlalchemy import MetaData

# Metadata si se utilizan tablas
metadata = MetaData()

En SQLAlchemy las tablas se crean medinate las clases Columms:
* Columns ofrece distintos tipos de datos (Integer, Float, etc.)
* Columns ofrece relaciones entre tablas (Primary Key, Foreign Key, etc.)

In [0]:
from sqlalchemy import Column, Integer, String, Table

# Definicion como tablas     
users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String),
              Column('password', String))                     

SQLAlchemy puede crear las tablas 

In [0]:
metadata.create_all(engine)

2019-06-04 07:47:17,002 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-06-04 07:47:17,004 INFO sqlalchemy.engine.base.Engine ()
2019-06-04 07:47:17,008 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-06-04 07:47:17,010 INFO sqlalchemy.engine.base.Engine ()
2019-06-04 07:47:17,012 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-06-04 07:47:17,013 INFO sqlalchemy.engine.base.Engine ()
2019-06-04 07:47:17,016 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2019-06-04 07:47:17,017 INFO sqlalchemy.engine.base.Engine ()
2019-06-04 07:47:17,018 INFO sqlalchemy.engine.base.Engine COMMIT


##### SQL:

```SQL
CREATE TABLE users (
    id INTEGER NOT NULL, name VARCHAR,
    fullname VARCHAR,
    password VARCHAR,
    PRIMARY KEY (id)
```

### Relationships 

Multiples tablas pueden ser realionadas usando la función `ForeignKey`.

In [0]:
from sqlalchemy import ForeignKey
    
addresses = Table('addresses', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('user_id', None, ForeignKey('users.id')),
                  Column('email_address', String, nullable=False))

metadata.create_all(engine)

### Inicio de Session

In [0]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

#Inicio de la sesion
session = Session()

### Inserción de datos

In [0]:
paco_user = users.insert().values(name='Paco', fullname='Francisco', password='25051970')

Datos como diccionario

In [0]:
paco_user.compile().params

{'fullname': 'Francisco', 'name': 'Paco', 'password': '25051970'}

Consolidación de datos

In [0]:
# Se instancia una conexión
conn = engine.connect()

result = conn.execute(paco_user)

In [0]:
result.inserted_primary_key

[1]

### Multiples inserciones

In [0]:
result = conn.execute(users.insert(), [
    {'name': 'Carla', 'fullname': 'Carla', 'password': '$5&02qq'},
    {'name': 'Manu', 'fullname': 'Manuela', 'password': 'lqoeoeee'},
    {'name': 'Lu', 'fullname': 'Lucia', 'password': 'fjakclff'}
])

result1 = conn.execute(addresses.insert(), [
    {'user_id': 2, 'email_address': 'carla1?22@gmail.com'},
    {'user_id': 4, 'email_address': 'luci.mendo@gmail.com'},
    {'user_id': 1, 'email_address': 'paco84@hotmail.com'}
])

### Update

In [0]:
up = users.update().where(users.c.name == 'Lu').\
        values(password='iqq@3399')

conn.execute(up)

In [0]:
from sqlalchemy import bindparam

up = users.update().\
             where(users.c.fullname == bindparam('oldfullname')).\
             values(fullname=bindparam('newfullname'))

conn.execute(up, [
     {'oldfullname':'Francisco', 'newfullname':'Francisco Javier'},
     ])

Actualización con diccionarios

In [0]:
up =  addresses.update().\
        values({
            addresses.c.email_address:'carlota12@gmail.com'
        }).\
        where(addresses.c.email_address.startswith('ca%'))

conn.execute(up)

### Select

In [0]:
from sqlalchemy.sql import select

# Obtenemos los usuarios
s = select([users])
result = conn.execute(s)

In [0]:
for row in result:
    print(row)

(1, 'Paco', 'Francisco', '25051970')
(2, 'Carla', 'Carla', '$5&02qq')
(3, 'Manu', 'Manuela', 'lqoeoeee')
(4, 'Lu', 'Lucia', 'iqq@3399')


SQLAlchemy tiene también métodos fetchone() y fetchall() 

In [0]:
result = conn.execute(s)
row = result.fetchone()

In [0]:
for elem in row:
    print(elem)

1
Paco
Francisco
25051970


In [0]:
rows = result.fetchall()

In [0]:
for elem in rows:
    print(elem)

(2, 'Carla', 'Carla', '$5&02qq')
(3, 'Manu', 'Manuela', 'lqoeoeee')
(4, 'Lu', 'Lucia', 'iqq@3399')


Con la el atributo `c` se puede acceder a una columna en concreto

In [0]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)

for row in result:
    print(row)

('Paco', 'Francisco')
('Carla', 'Carla')
('Manu', 'Manuela')
('Lu', 'Lucia')


Selección de varias tablas

In [0]:
for row in conn.execute(select([users, addresses])):
    print(row)

(1, 'Paco', 'Francisco', '25051970', 1, 2, 'carlota12@gmail.com')
(1, 'Paco', 'Francisco', '25051970', 2, 4, 'luci.mendo@gmail.com')
(1, 'Paco', 'Francisco', '25051970', 3, 1, 'paco84@hotmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 1, 2, 'carlota12@gmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 2, 4, 'luci.mendo@gmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 3, 1, 'paco84@hotmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 1, 2, 'carlota12@gmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 2, 4, 'luci.mendo@gmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 3, 1, 'paco84@hotmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 1, 2, 'carlota12@gmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 2, 4, 'luci.mendo@gmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 3, 1, 'paco84@hotmail.com')


##### SQL:

```SQL
SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address
FROM users, addresses
```

In [0]:
# Para mejorar la sentencia anterior
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in conn.execute(s):
    print(row)

(2, 'Carla', 'Carla', '$5&02qq', 1, 2, 'carlota12@gmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 2, 4, 'luci.mendo@gmail.com')
(1, 'Paco', 'Francisco', '25051970', 3, 1, 'paco84@hotmail.com')


##### SQL:

```SQL
SELECT users.id, users.name, users.fullname, addresses.id,
   addresses.user_id, addresses.email_address
FROM users, addresses
WHERE users.id = addresses.user_id
```

Uso de sentencias SQL

In [0]:
from sqlalchemy.sql import text

s = text("SELECT users.id, addresses.id, users.id, "
     "users.name, addresses.email_address AS email "
     "FROM users JOIN addresses ON users.id=addresses.user_id "
     "WHERE users.id = 1").columns(
        users.c.id,
        addresses.c.id,
        addresses.c.user_id,
        users.c.name,
        addresses.c.email_address
     )

result = conn.execute(s)

In [0]:
result.fetchall()

[(1, 3, 1, 'Paco', 'paco84@hotmail.com')]

Una vez realizadas las consultas cerramos el objeto

In [0]:
result.close()

### Conjunctions

In [0]:
from sqlalchemy.sql import and_, or_, not_

sent = and_(
    users.c.name.like('c%'),
    users.c.id == addresses.c.user_id,
    or_(
        addresses.c.email_address == 'carla1?22@gmail.com',
        addresses.c.email_address == 'luci.mendo@gmail.com' 
    ),
    not_(users.c.id > 3)
    )
print(sent)

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [0]:
s = select([(users.c.fullname +
               ", " + addresses.c.email_address)]).\
        where(
           and_(
               users.c.id == addresses.c.user_id,
               or_(
                  addresses.c.email_address.like('%@gmail.com'),
                  addresses.c.email_address.like('%@hotmail.com')
               )
           )
        )
conn.execute(s).fetchall()

[('Carla, carlota12@gmail.com',),
 ('Lucia, luci.mendo@gmail.com',),
 ('Francisco, paco84@hotmail.com',)]

### Funciones

In [0]:
from sqlalchemy.sql import func

In [0]:
conn.execute(
    select([
            func.max(addresses.c.email_address, type_=String)
           ])
     ).scalar() # Primer elemento del primer resultado

'paco84@hotmail.com'

In [0]:
conn.execute(
    select([
            func.min(addresses.c.user_id)
           ])
     ).scalar()

1

### Alias

In [0]:
a = addresses.alias()
s = select([users]).\
        where(and_(
            users.c.id == a.c.user_id,
            a.c.email_address == 'paco84@hotmail.com'
        ))

conn.execute(s).fetchall()

[(1, 'Paco', 'Francisco', '25051970')]

### Order_by

In [0]:
s = select([users.c.name]).order_by(users.c.name)

conn.execute(s).fetchall()

[('Carla',), ('Lu',), ('Manu',), ('Paco',)]

In [0]:
# En orden descendente
s = select([users.c.name]).order_by(users.c.name.desc())

conn.execute(s).fetchall()

[('Paco',), ('Manu',), ('Lu',), ('Carla',)]

### Group_by

In [0]:
s = select([users.c.name, func.count(addresses.c.id)]).\
             select_from(users.join(addresses)).\
             group_by(users.c.name)

conn.execute(s).fetchall()

[('Carla', 1), ('Lu', 1), ('Paco', 1)]

### Having

In [0]:
s = select([users.c.name, func.count(addresses.c.id)]).\
             select_from(users.join(addresses)).\
             group_by(users.c.name).\
             having(func.length(users.c.name) > 4)

conn.execute(s).fetchall()

[('Carla', 1)]

### Limit y offset

In [0]:
s = select([users.c.name, addresses.c.email_address]).\
             select_from(users.join(addresses)).\
             limit(2).offset(1)

conn.execute(s).fetchall()

[('Lu', 'luci.mendo@gmail.com'), ('Paco', 'paco84@hotmail.com')]

### Union 

In [0]:
from sqlalchemy.sql import union

u = union(
    addresses.select().
             where(addresses.c.email_address.like('%@gmail.com')),
    addresses.select().
             where(addresses.c.email_address.like('%@hotmail.com')),
    ).order_by(addresses.c.email_address)

conn.execute(u).fetchall()

[(1, 2, 'carlota12@gmail.com'),
 (2, 4, 'luci.mendo@gmail.com'),
 (3, 1, 'paco84@hotmail.com')]

### Except

In [0]:
from sqlalchemy.sql import except_

# Devuelve las filas del primer SELECT que no son devuletas por el segundo SELECT 
e = except_(
    addresses.select().
             where(addresses.c.email_address.like('%@gmail.com')),
    addresses.select().
             where(addresses.c.email_address.like('%@hotmail.com')),
    ).order_by(addresses.c.email_address)

conn.execute(e).fetchall()

[(1, 2, 'carlota12@gmail.com'), (2, 4, 'luci.mendo@gmail.com')]

### Join

In [0]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [0]:
print(users.join(addresses, addresses.c.email_address.like(users.c.name + '%')))

users JOIN addresses ON addresses.email_address LIKE users.name || :name_1


In [0]:
s = select([users.c.fullname]).select_from(
        users.join(addresses,
        addresses.c.email_address.like(users.c.name + '%'))
        )

conn.execute(s).fetchall()

[('Francisco',), ('Lucia',)]

### Delete

In [0]:
conn.execute(addresses.delete())

In [0]:
conn.execute(users.delete().where(users.c.name > 'm'))

## Uso de Clases

### Descripción de tablas con clases 

In [0]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [0]:
from sqlalchemy import Column, Integer, String
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
  __tablename__ = 'users'
  
  id = Column(Integer, primary_key=True)
  name = Column(String)
  fullname = Column(String)
  nickname = Column(String)
  
  def __init__(self, name, fullname, nickname):
    self.name = name
    self.fullname = fullname
    self.nickname = nickname
  
  
class Address(Base):
  __tablename__ = 'addresses'
  
  id = Column(Integer, primary_key=True)
  email_address = Column(String, nullable=False)
  user_id = Column(Integer, ForeignKey('users.id'))
  
  user = relationship("User", backref="addresses")
  
  def __init__(self, email_address, user):
    self.email_address = email_address
    self.user = user

Connexión a la base de datos

In [0]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:',echo=False)

### Create

In [0]:
Base.metadata.create_all(engine) 

###  Creación de sesión



In [0]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

### Insercción de datos

In [0]:
user_1 = User('Carla', 'Carla', '$5&02qq')
user_2 = User('Manu', 'Manuela', 'lqoeoeee')
user_3 = User('Lu', 'Lucia', 'password')

address_1 = Address('carla1?22@gmail.com', user_1)
address_2 = Address('luci.mendo@gmail.com', user_2)
address_3 = Address('paco84@hotmail.com', user_3)

session.add(user_1)
session.add(user_2)
session.add(user_3)

session.add(address_1)
session.add(address_2)
session.add(address_3)

# O
# session.add_all([user_1, user_2, user_3, address_1, address_3, address_3])
session.commit()

### Update

In [0]:
address_3.email_address = 'paco85@hotmail.com'

#Comprobamos si el objeto está en la sesión
print(address_3 in session)
session.commit()

True


#### Cierre de sesión



In [0]:
session.close()

## Querying

In [0]:
for instance in session.query(User).order_by(User.id):
  print(instance.name, instance.fullname)

Carla Carla
Manu Manuela
Lu Lucia


In [0]:
for row in session.query(User, User.name):
  print(row.User, row.name)

<__main__.User object at 0x7fe789f45a58> Carla
<__main__.User object at 0x7fe789f45a90> Manu
<__main__.User object at 0x7fe789db2fd0> Lu


In [0]:
for user in session.query(User).\
        filter(User.name=='Lu').\
        filter(User.fullname=='Lucia'):
  print(user.name)

Lu


In [0]:
session.query(User).filter(User.name.like('%u')).order_by(User.id)[0].name

'Manu'

In [0]:
#Join
session.query(User).join(Address).filter(Address.email_address=='luci.mendo@gmail.com').all()

### Delete

In [0]:
session.delete(user_1)

In [0]:
session.query(User).filter_by(name='Paco').count()

2019-05-30 17:53:05,790 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-30 17:53:05,795 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2019-05-30 17:53:05,797 INFO sqlalchemy.engine.base.Engine (1,)
2019-05-30 17:53:05,801 INFO sqlalchemy.engine.base.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2019-05-30 17:53:05,804 INFO sqlalchemy.engine.base.Engine (1,)
2019-05-30 17:53:05,807 INFO sqlalchemy.engine.base.Engine UPDATE addresses SET user_id=? WHERE addresses.id = ?
2019-05-30 17:53:05,809 INFO sqlalchemy.engine.base.Engine (None, 1)
2019-05-30 17:53:05,810 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = ?
2019-05-30 17:53:05,812 INFO sqlalchemy.engine.base.Engine (1,)
2019-05-3

0

## Ejercicios

1. Defina dos tablas (users y accounts) haciendo uso de SQLAlchemy con los siguientes atributos:
    * users: 
        - user_id: tipo Integer, es primary_key
        - username: tipo String, no pueder ser nulo, tiene que ser único
        - email_address: tipo String, no puede ser nulo
        - phone: tipo String, no puede ser nulo
        - password: tipo String, no puede ser nulo
        - created_on: tipo DateTime, tien por defecto la hora de creación
    * accounts:
        - accounts_id: tipo Integer, es primary_key
        - user_id: es ForeignKey de users.user_id
        - iban: tipo String
        - bic: tipo String
        - money: tipo Numeric con 2 decimales
        
2. Cree las tablas anteriormente definidas con ayuda de un conector SQLite en un fichero de nombre `test.db`. 
3. Introducta al menos 3 valores por cada tabla.
4. Lea todos los valores de las tablas para comprobar se han introducido de forma correcta. 
5. Ordene los usuarios por la cantidad de dinero en cuenta 